<a href="https://colab.research.google.com/github/NikethnaSri-AI/Aanseaa_Technologies_Internship/blob/main/Task%201/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install emoji


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 14.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import re
import emoji
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [4]:
import tensorflow as tf
df=pd.read_csv('Tweets.csv')

confidence_threshold = 0.6
df = df.drop(df.query('airline_sentiment_confidence < @confidence_threshold').index, axis=0).reset_index(drop=True)

In [5]:
tweet_df = pd.concat([df['text'], df['airline_sentiment']], axis=1)
tweet_df.isna().sum().sum()

np.int64(0)

In [6]:
tweet_df['airline_sentiment'].value_counts()

,count
airline_sentiment,
negative,9113
neutral,2997
positive,2292


In [7]:
print(tweet_df)

                                                    text airline_sentiment
0                    @VirginAmerica What @dhepburn said.           neutral
1      @VirginAmerica I didn't today... Must mean I n...           neutral
2      @VirginAmerica it's really aggressive to blast...          negative
3      @VirginAmerica and it's a really big bad thing...          negative
4      @VirginAmerica seriously would pay $30 a fligh...          negative
...                                                  ...               ...
14397         @AmericanAir right on cue with the delays👌          negative
14398  @AmericanAir leaving over 20 minutes Late Flig...          negative
14399  @AmericanAir Please bring American Airlines to...           neutral
14400  @AmericanAir you have my money, you change my ...          negative
14401  @AmericanAir we have 8 ppl so we need 2 know h...           neutral

[14402 rows x 2 columns]


In [8]:
sentiment_ordering=['negative','neutral','positive']
tweet_df['airline_sentiment'] = tweet_df['airline_sentiment'].apply(lambda x: sentiment_ordering.index(x))
print(tweet_df)

                                                    text  airline_sentiment
0                    @VirginAmerica What @dhepburn said.                  1
1      @VirginAmerica I didn't today... Must mean I n...                  1
2      @VirginAmerica it's really aggressive to blast...                  0
3      @VirginAmerica and it's a really big bad thing...                  0
4      @VirginAmerica seriously would pay $30 a fligh...                  0
...                                                  ...                ...
14397         @AmericanAir right on cue with the delays👌                  0
14398  @AmericanAir leaving over 20 minutes Late Flig...                  0
14399  @AmericanAir Please bring American Airlines to...                  1
14400  @AmericanAir you have my money, you change my ...                  0
14401  @AmericanAir we have 8 ppl so we need 2 know h...                  1

[14402 rows x 2 columns]


In [9]:
ps=PorterStemmer()
def process_tweet(tweet):
  new_tweet = tweet.lower()
  new_tweet = re.sub(r'@\w+', '', new_tweet)
  new_tweet = re.sub(r'#', '', new_tweet)
  new_tweet = re.sub(r':', '', emoji.demojize(new_tweet))
  new_tweet = re.sub(r'http\S+', '', new_tweet)
  new_tweet = re.sub(r'\$\S+', 'dollar', new_tweet)
  new_tweet = re.sub(r'[^a-z0-9\s]', '', new_tweet)
  new_tweet = re.sub(r'[0-9]+', 'number', new_tweet)
  new_tweet = new_tweet.split(" ")
  new_tweet = list(map(lambda x: ps.stem(x), new_tweet))
  new_tweet = list(map(lambda x: x.strip(), new_tweet))
  if '' in new_tweet:
    new_tweet.remove('')
  return new_tweet

In [10]:
tweets = tweet_df['text'].apply(process_tweet)
labels=np.array(tweet_df['airline_sentiment'])
print(tweets)

0                                           [what, , said]
1        [i, didnt, today, must, mean, i, need, to, tak...
2        [it, realli, aggress, to, blast, obnoxi, enter...
3         [and, it, a, realli, big, bad, thing, about, it]
4        [serious, would, pay, dollar, a, flight, for, ...
                               ...                        
14397              [right, on, cue, with, the, delayshand]
14398    [leav, over, number, minut, late, flight, no, ...
14399    [pleas, bring, american, airlin, to, blackberr...
14400    [you, have, my, money, you, chang, my, flight,...
14401    [we, have, number, ppl, so, we, need, number, ...
Name: text, Length: 14402, dtype: object


In [11]:
vocab = set()
for tweet in tweets:
    for word in tweet:
        if word not in vocab:
            vocab.add(word)

vocab_length = len(vocab)
max_seq_length = max(len(tweet) for tweet in tweets)

print("Vocab length:", vocab_length)
print("Max sequence length:", max_seq_length)


Vocab length: 10286
Max sequence length: 35


In [12]:
tokenizer = Tokenizer(num_words=vocab_length)
tokenizer.fit_on_texts(tweets)
sequences = tokenizer.texts_to_sequences(tweets)
word_index = tokenizer.word_index
model_inputs = pad_sequences(sequences, maxlen=max_seq_length, padding='post')
print(sequences)

[[48, 2, 217], [4, 190, 100, 735, 352, 4, 61, 1, 112, 150, 179], [15, 137, 2802, 1, 1991, 3385, 767, 14, 17, 1711, 986, 65, 57, 19, 473, 2177], [11, 15, 7, 137, 457, 212, 287, 84, 15], [415, 77, 201, 108, 7, 6, 9, 82, 20, 190, 19, 29, 4502, 137, 3, 122, 212, 287, 84, 58, 1518], [174, 1321, 305, 43, 4, 58, 1834, 29, 2424, 4503, 209, 66, 462, 2], [137, 104, 7, 2425, 1379, 9, 2803, 325, 2178, 4504, 67, 2], [191, 4, 4505, 38, 4, 39, 1380], [15, 23, 335, 11, 205, 41, 32, 348, 17, 177, 139, 1, 18], [124, 8, 109, 20, 1992, 13, 3, 502, 1264, 421, 16, 1265, 1451, 1835, 5], [4, 1116, 639, 3386, 46, 176, 215, 163, 4506, 4507, 1380], [29, 13, 525, 7, 120, 357, 219, 224, 84, 12, 584, 179, 1, 2, 65, 4, 332, 127, 909, 10, 12, 333, 179, 252, 1608], [2, 51, 58, 17, 1609, 4508, 968, 111, 125, 112, 69, 3, 987, 462, 31, 113, 2], [21], [4509, 349, 13, 73, 883], [46, 884, 9, 12, 159, 696, 768, 6, 300, 1, 736, 119, 534, 248, 30, 120, 287, 84, 865, 1016, 4510], [2, 4, 617, 31, 483, 1, 353, 132, 200, 11, 326, 

In [13]:
print(word_index)

{'to': 1, '': 2, 'the': 3, 'i': 4, 'number': 5, 'flight': 6, 'a': 7, 'you': 8, 'for': 9, 'on': 10, 'and': 11, 'my': 12, 'is': 13, 'in': 14, 'it': 15, 'of': 16, 'your': 17, 'me': 18, 'have': 19, 'that': 20, 'thank': 21, 'get': 22, 'wa': 23, 'not': 24, 'with': 25, 'at': 26, 'no': 27, 'be': 28, 'thi': 29, 'but': 30, 'from': 31, 'hour': 32, 'can': 33, 'are': 34, 'cancel': 35, 'we': 36, 'help': 37, 'now': 38, 'do': 39, 'servic': 40, 'an': 41, 'delay': 42, 'time': 43, 'just': 44, 'custom': 45, 'so': 46, 'been': 47, 'what': 48, 'will': 49, 'call': 50, 'im': 51, 'bag': 52, 'wait': 53, 'up': 54, 'plane': 55, 'out': 56, 'they': 57, 'fli': 58, 'our': 59, 'hold': 60, 'need': 61, 'us': 62, 'when': 63, 'how': 64, 'amp': 65, 'go': 66, 'there': 67, 'if': 68, 'all': 69, 'whi': 70, 'cant': 71, 'tri': 72, 'still': 73, 'one': 74, 'pleas': 75, 'airlin': 76, 'would': 77, 'day': 78, 'gate': 79, 'flightl': 80, 'back': 81, 'seat': 82, 'had': 83, 'about': 84, 'book': 85, 'dont': 86, 'ha': 87, 'or': 88, 'as': 89

In [14]:
model_inputs.shape

(14402, 35)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(model_inputs, labels, train_size=0.7, random_state=22)

In [20]:
embedding_dim = 32

inputs = tf.keras.Input(shape=(max_seq_length,))
embedding = tf.keras.layers.Embedding(
    input_dim=len(tokenizer.word_index) + 1,
    output_dim=embedding_dim,
    input_length=max_seq_length
)(inputs)

flatten = tf.keras.layers.Flatten()(embedding)
outputs = tf.keras.layers.Dense(3, activation='softmax')(flatten)
model = tf.keras.Model(inputs, outputs)


In [22]:
outputs=tf.keras.layers.Dense(3, activation='softmax')(flatten)
model=tf.keras.Model(inputs, outputs)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

batch_size=32
epochs=100
history=model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau()
    ]
)


Epoch 1/100
252/252 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6887 - loss: 0.7130 - val_accuracy: 0.7913 - val_loss: 0.5275 - learning_rate: 0.0010
Epoch 2/100
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8940 - loss: 0.3239 - val_accuracy: 0.7987 - val_loss: 0.5074 - learning_rate: 0.0010
Epoch 3/100
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9338 - loss: 0.2270 - val_accuracy: 0.8012 - val_loss: 0.5230 - learning_rate: 0.0010
Epoch 4/100
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9522 - loss: 0.1704 - val_accuracy: 0.7982 - val_loss: 0.5423 - learning_rate: 0.0010
Epoch 5/100
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9694 - loss: 0.1242 - val_accuracy: 0.7878 - val_loss: 0.5647 - learning_rate: 0.0010
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 2.


In [23]:
def evaluate_sample_tweet(model, tokenizer, max_seq_length, sentiment_ordering):
    tweet = input("Enter your tweet: ")
    processed_tweet = process_tweet(tweet)
    sequence = tokenizer.texts_to_sequences([processed_tweet])
    inputs = pad_sequences(sequence, maxlen=max_seq_length, padding='post')
    prediction = model.predict(inputs)
    predicted_label_index = np.argmax(prediction)

    predicted_label = sentiment_ordering[predicted_label_index]
    accuracy = prediction[0][predicted_label_index]
    print(f"Predicted Sentiment: {predicted_label}")
    print(f"Accuracy: {accuracy:.2f}")


In [24]:
evaluate_sample_tweet(model, tokenizer, max_seq_length, ['negative','neutral','positive'])

Enter your tweet: flight was delayed by 3hrs!! 😡 😡
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Predicted Sentiment: negative
Accuracy: 0.91


In [25]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(confusion_matrix(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes, target_names=sentiment_ordering))

136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[2483  237   77]
 [ 275  523   71]
 [ 106   85  464]]
              precision    recall  f1-score   support

    negative       0.87      0.89      0.88      2797
     neutral       0.62      0.60      0.61       869
    positive       0.76      0.71      0.73       655

    accuracy                           0.80      4321
   macro avg       0.75      0.73      0.74      4321
weighted avg       0.80      0.80      0.80      4321

